## Day3 - Conversational AI - aka ChatBot!

In [1]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

C:\Users\hp\.conda\envs\llms1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key not set


In [3]:
# Initialize 

openai = OpenAI()
MODEL='gpt-4o-mini'

In [4]:
system_message = "You are a helpful assistant"

## Please read this! A change from the video:
In the video, I explain how we now need to write a function called:

chat(message, history)

Which expects to receive history in a particular format, which we need to map to the OpenAI format before we call OpenAI:

[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
But Gradio has been upgraded! Now it will pass in history in the exact OpenAI format, perfect for us to send straight to OpenAI.

So our work just got easier!

We will write a function chat(message, history) where:
message is the prompt to use
history is the past conversation, in OpenAI format

We will combine the system message, history and latest message, then call OpenAI.

In [5]:
# old

def chat(message, history):
    messages = [{"role":"system", "content":system_message}]
    for user_message, assistant_message in history:
        messages.append({"role":"user", "content":user_message})
        messages.append({"role":"assistant", "content":assistant_message})
    messages.append({"role":"user", "content":message})

    print("History is : ")
    print(history)
    print("And messages is :")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response=""
    for chunk in stream:
        response+=chunk.choices[0].delta.content or ''
        yield response

### And then enter Gradio's magic!

In [10]:
gr.ChatInterface(fn=chat).launch()

C:\Users\hp\.conda\envs\llms1\Lib\site-packages\gradio\components\chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.


In [11]:
# new
# Simpler than in my video - we can easily create this function that calls OpenAI
# It's now just 1 line of code to prepare the input to OpenAI!

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    print("History is:")
    print(history)
    print("And messages is:")
    print(messages)

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

### And then enter Gradio's magic!

In [14]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


In [15]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [16]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [17]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7884

To create a public link, set `share=True` in `launch()`.


In [18]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [19]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.


In [20]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [21]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7886

To create a public link, set `share=True` in `launch()`.


In [22]:
# Fixed a bug in this function brilliantly identified by student Gabor M.!
# I've also improved the structure of this function

def chat(message, history):

    relevant_system_message = system_message
    if 'belt' in message:
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [23]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7887

To create a public link, set `share=True` in `launch()`.


In [ ]:
# Constants OLLAMA Local Setup
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}
MODEL = "llama3.2:1b"

# Create a messages list using the same format that we used for OpenAI
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

payload = {
   "model": MODEL,
   "messages": messages,
   "stream": True
}

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True)

In [31]:
import requests
from typing import Generator

import requests
from typing import Generator

def chat_with_ollama(user_message: str, system_prompt: str = "") -> Generator[str, None, None]:
    """
    Send a chat message to Ollama and stream the response.
    
    Args:
        user_message (str): The user's message to send to Ollama
        system_prompt (str, optional): System prompt to set context. Defaults to empty string.
    
    Returns:
        Generator[str, None, None]: A generator yielding response chunks
        
    Raises:
        requests.exceptions.RequestException: If the API request fails
    """
    # Constants
    OLLAMA_API = "http://localhost:11434/api/chat"
    HEADERS = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    MODEL = "llama2:1b"
    
    # Create messages list
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]
    
    # Prepare payload
    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": True
    }
    
    try:
        # Make API request
        response = requests.post(
            OLLAMA_API,
            json=payload,
            headers=HEADERS
            # stream=True
        )
        response.raise_for_status()
        
        # Stream the response
        for line in response.iter_lines():
            if line:
                yield line.decode('utf-8')
                
    except requests.exceptions.RequestException as e:
        raise Exception(f"Failed to communicate with Ollama: {str(e)}")

In [32]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [33]:
user_message = "I want to buy a bow-tie."

In [34]:
response = chat_with_ollama(user_message, system_message)

In [35]:
print(response)

<generator object chat_with_ollama at 0x0000016A4F964890>


In [41]:
import requests
from typing import Generator
import json

def chat_with_ollama(user_message: str, system_prompt: str = "") -> Generator[str, None, None]:
    """
    Send a chat message to Ollama and stream the response.
    """
    OLLAMA_API = "http://localhost:11434/api/chat"
    HEADERS = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    MODEL = "llama3.2:1b"

    system_prompt = "You are a helpful assistant in a clothes store. You should try to gently encourage \
    the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
    For example, if the customer says 'I'm looking to buy a hat', \
    you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
    Encourage the customer to buy hats if they are unsure what to get."
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]
    
    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": True
    }
    
    try:
        response = requests.post(
            OLLAMA_API,
            json=payload,
            headers=HEADERS,
            stream=True
        )
        response.raise_for_status()
        
        for line in response.iter_lines():
            if line:
                yield line.decode('utf-8')
                
    except requests.exceptions.RequestException as e:
        raise Exception(f"Failed to communicate with Ollama: {str(e)}")

def display_chat_response():
    """
    Example function showing how to use chat_with_ollama and display the response.
    """
    # Example messages to test
    messages_to_test = [
        "What is the price of shoes?",
        "Tell me how much discount is today.",
        "What's the weather like today?"
    ]
    
    for user_message in messages_to_test:
        print(f"\nUser: {user_message}")
        print("Assistant: ", end='')
        
        try:
            full_response = ""
            for chunk in chat_with_ollama(user_message):
                try:
                    # Parse the JSON chunk
                    response_data = json.loads(chunk)
                    
                    # Extract the response content
                    if 'message' in response_data:
                        content = response_data['message'].get('content', '')
                        print(content, end='', flush=True)
                        full_response += content
                        
                except json.JSONDecodeError:
                    print(f"Error parsing chunk: {chunk}")
                    continue
                    
            print()  # New line after response
            
        except Exception as e:
            print(f"Error: {e}")
            continue

display_chat_response()


User: What is the price of shoes?
We have a great selection of shoes at discounted prices. As it happens, all our dress shoes are 50% off, and our heels are 60% off. Would you like me to show you some options in those styles that might be up your alley? By the way, we also have a few pairs of boots that are on sale - they're a great price for quality craftsmanship.

User: Tell me how much discount is today.
We're having a special promotion today, and I think you might be interested in checking out some of our sale items. Our hats are 60% off, which means you can get a fantastic deal on them. Plus, many of the other tops, dresses, and pants are also being discounted by 50%. Would you like me to show you some of the styles that are currently on sale?

User: What's the weather like today?
I'd be happy to help you check the weather forecast, but I don't have real-time access to current conditions. However, I can suggest some options for you to find out the latest weather update.

You can 